In [ ]:
#!M
# !pip install ../input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl > /dev/null 2>&1
# %pip install --upgrade pip
# %pip install -r /home/jupyter/work/resources/riiidNew/requirements.txt --upgrade
# %pip install scipy  --ignore-installed scipy --upgrade
print("ok")

In [1]:
#!L
# if str(Path.home()) == "/root":  # DATASPHERE
import sys
import gc
from pathlib import Path
sys.path.append(str(Path.cwd()/'riiidNew'))
from pathlib import Path
import numpy as np
import pandas as pd
import pyarrow
from collections import defaultdict
from catboost import CatBoostClassifier, Pool
from preprocess import preprocess_train_data
from catboost_bayesian_search import bayesian_catboost_searchCV, param_adjust_dtypes
import config

# Error handling, ignore all
np.seterr(divide = 'ignore', invalid = 'ignore')


if str(Path.home()) == "/home/sergey":   # Home computer
    kaggle_path = Path.cwd()/"kaggle_tmp/"
    questions_df = pd.read_csv('/mnt/data30G/2020riiid/questions.csv', usecols = [0, 3], dtype = {'question_id': 'int16', 'part': 'int8'})
    test_df = pd.read_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_valid_1e4.feather", columns=dtypes.keys())
    train_df = pd.read_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_train_1e5.feather", columns=dtypes.keys())
    # test_df = pd.read_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_valid_1e4.feather", columns=dtypes.keys())
    # train_df = pd.read_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_train.feather", columns=dtypes.keys())
    # test_df = pd.read_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_valid.feather", columns=dtypes.keys())

elif str(Path.home()) == "/root":   # DATASPHERE
    
    path = Path.cwd()/"riiidNew"/"data1"
    kaggle_path = Path.cwd()/"riiidNew"/"kaggle"
    questions_df = pd.read_csv(path/'questions.csv', usecols = [0, 3], dtype = {'question_id': 'int16', 'part': 'int8'})
    train_df = pd.read_feather(path/"train.feather", columns=config.dtypes.keys())
    # train_df = pd.read_feather(path/"train_5e6.feather", columns=config.dtypes.keys())
    # train_df = pd.read_feather(path/"train_1e5.feather", columns=config.dtypes.keys())
    # train_df = pd.read_pickle(path/'cv2_train.pickle.zip').astype(config.dtypes, errors="ignore")
    # test_df = pd.read_pickle(path/"cv2_valid.pickle.zip").astype(config.dtypes, errors="ignore")

# print(f"train_df shape = {train_df.shape}")

# Preprocess for CV
train_user_agg, train_content_agg, train_df = preprocess_train_data(train_df, questions_df, config.target, config.dtypes)
# test_user_agg, test_content_agg, test_df = preprocess_train_data(test_df, questions_df, config.target, config.dtypes)
print("Preprocess for CV ok")

# Training and validating data
train_set = Pool(train_df[config.features], label = train_df[config.target], cat_features=config.cat_features)
# val_set = Pool(test_df[features], label = test_df[target])

print("optimizer_maxCV ..")
optimizer_maxCV = bayesian_catboost_searchCV(
    train_set,
    prior_params=config.prior_params,
    pds=config.pds, pds_dtypes=config.pds_dtypes,
    init_points=3, n_iter=25, verbose=False,
)
print("optimizer_maxCV ..")
print(optimizer_maxCV)

params = param_adjust_dtypes(config.prior_params, 
                             config.pds_dtypes, 
                             optimizer_maxCV["params"])
print("params")
print(params)

# pds_fitted = {key: config.pds_dtypes[key](val) for key, val in optimizer_maxCV["params"].items()}
# params = prior_params.copy()
# params.update(pds_fitted)

# Ratio is 6/24 = 25%
# valid_df = train_df.groupby('user_id').tail(6)
# train_df.drop(valid_df.index, inplace = True)

import pickle

user_sum_dict = train_user_agg['sum'].astype('int32').to_dict(defaultdict(int))
user_count_dict = train_user_agg['count'].astype('int32').to_dict(defaultdict(int))
content_sum_dict = train_content_agg['sum'].astype('int32').to_dict(defaultdict(int))
content_count_dict = train_content_agg['count'].astype('int32').to_dict(defaultdict(int))

for filename, dic in zip(["user_sum_dict", "user_count_dict", "content_sum_dict", "content_count_dict"],
                         [user_sum_dict, user_count_dict, content_sum_dict, content_count_dict]):
    with open(f'{kaggle_path}/{filename}.pickle', 'wb') as handle:
        pickle.dump(dic, handle)

# del train_df, user_sum_dict, train_user_agg, user_count_dict
# del content_sum_dict, train_content_agg, content_count_dict
# gc.collect()

# Obtain new trai_df and validation test_df
# train_df = pd.read_feather(path/"cv2_train.feather", columns=config.dtypes.keys())

# train_df = pd.read_pickle(path/'cv2_train.pickle.zip').astype(config.dtypes, errors="ignore").reset_index()
# train_df = train_df.iloc[:int(1e5)]

# test_df = pd.read_feather(path/"cv2_test.feather", columns=config.dtypes.keys())
# test_df = pd.read_pickle(path/"cv2_valid.pickle.zip").astype(config.dtypes, errors="ignore").reset_index()
# test_df = test_df.iloc[:int(1e4)]

print("cv2_train.feather, cv2_test.feather ok!") 


# Preprocess for model fitting
# train_user_agg, train_content_agg, train_df = preprocess_train_data(train_df, questions_df, config.target, config.dtypes)
# test_user_agg, test_content_agg, test_df = preprocess_train_data(test_df, questions_df, config.target, config.dtypes)
print("Preprocess for model fitting ok")

# Ratio is 6/24 = 25%
test_df = train_df.groupby('user_id').tail(6)
train_df.drop(test_df.index, inplace = True)

# Training and validating data
train_set = Pool(train_df[config.features], label = train_df[config.target], cat_features=config.cat_features)
val_set = Pool(test_df[config.features], label = test_df[config.target], cat_features=config.cat_features)


# val_set = Pool(valid_df[features], label = valid_df[target])

# Model definition
# params = {
#     'loss_function': 'Logloss', 
#     'eval_metric': 'AUC', 
#     'custom_metric': 'AUC:hints=skip_train~false', 
#     'task_type': 'GPU', 'grow_policy': 'Lossguide', 
#     'iterations': 2000, 'learning_rate': 0.03, 
#     'random_seed': 0, 'bootstrap_type': 'Bayesian', 
#     'l2_leaf_reg': 463.75241951019393, 
#     'depth': 23, 'max_leaves': 21, 
#     'border_count': 163, 
#     'verbose': 250, 
#     'od_type': 'Iter', 
#     'od_wait': 50, 
#     'bagging_temperature': 2.055885684521886}
params["learning_rate"] = 4e-3
params['iterations'] = 20000

model = CatBoostClassifier(**params)

# Fitting
model.fit(train_set, eval_set = val_set, use_best_model = True)
print(f"kaggle_path {kaggle_path}")

model.save_model(f"{kaggle_path/'catboost.model'}")

print("model.get_best_score()")
print(model.get_best_score())

Preprocess for CV ok
optimizer_maxCV ..
0:	learn: 0.7394486	test: 0.7393291	best: 0.7393291 (0)
250:	learn: 0.7480574	test: 0.7477063	best: 0.7477063 (250)
500:	learn: 0.7485925	test: 0.7480717	best: 0.7480717 (500)
750:	learn: 0.7489159	test: 0.7482441	best: 0.7482441 (750)
1000:	learn: 0.7491736	test: 0.7483632	best: 0.7483632 (1000)
1250:	learn: 0.7494008	test: 0.7484592	best: 0.7484592 (1250)
1500:	learn: 0.7495914	test: 0.7485240	best: 0.7485240 (1500)
1750:	learn: 0.7497675	test: 0.7485799	best: 0.7485799 (1750)
1999:	learn: 0.7499298	test: 0.7486264	best: 0.7486264 (1999)	total: 9m 27s	remaining: 0us
0:	learn: 0.7383062	test: 0.7382177	best: 0.7382177 (0)
250:	learn: 0.7476609	test: 0.7473773	best: 0.7473773 (250)
500:	learn: 0.7481902	test: 0.7477813	best: 0.7477813 (500)
750:	learn: 0.7484849	test: 0.7479628	best: 0.7479628 (750)


/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:588: UserWarning: The following variables cannot be serialized: train_set
  warnings.warn(message)


KeyboardInterrupt: 


{'loss_function': 'Logloss', 'eval_metric': 'AUC', 'task_type': 'GPU', 'grow_policy': 'Lossguide', 'iterations': 15000, 'learning_rate': 0.01, 'random_seed': 0, 'l2_leaf_reg': 0.1, 'depth': 8, 'border_count': 128, 'verbose': 150, 'od_type': 'Iter', 'od_wait': 50}
bestTest = 0.7371392846

bestTest = 0.7370638549
bestTest = 0.7364509702


# Inference

In [ ]:
#!M
import pickle

user_sum_dict = train_user_agg['sum'].astype('int32').to_dict(defaultdict(int))
user_count_dict = train_user_agg['count'].astype('int32').to_dict(defaultdict(int))
content_sum_dict = train_content_agg['sum'].astype('int32').to_dict(defaultdict(int))
content_count_dict = train_content_agg['count'].astype('int32').to_dict(defaultdict(int))

for filename, dic in zip(["user_sum_dict", "user_count_dict", "content_sum_dict", "content_count_dict"],
                         [user_sum_dict, user_count_dict, content_sum_dict, content_count_dict]):
    with open(f'{kaggle_path}/{filename}.pickle', 'wb') as handle:
        pickle.dump(dic, handle)


In [ ]:
train_df.head()
del train_df
gc.collect()


In [ ]:
%%time
validaten_flg = True
if validaten_flg:
    from emulator import Iter_Valid
    iter_test = Iter_Valid(test_df,max_user=1000)
    predicted = []
    def set_predict(df):
        predicted.append(df)
else:
    import riiideducation
    env = riiideducation.make_env()
    iter_test = env.iter_test()
    set_predict = env.predict

In [ ]:
# cumcount = sum([len(df) for df in predicted])
# count = 0
# pbar = tqdm(total=cumcount)
# previous_test_df = None
# for (current_test, current_prediction_df) in iter_test:
#     count+=1
#     if previous_test_df is not None:
#         answers = eval(current_test["prior_group_answers_correct"].iloc[0])
#         responses = eval(current_test["prior_group_responses"].iloc[0])
#         previous_test_df['answered_correctly'] = answers
#         previous_test_df['user_answer'] = responses
#         # your feature extraction and model training code here
#     previous_test_df = current_test.copy()
#     current_test = current_test[current_test.content_type_id == 0]
#     # your prediction code here
#     current_test['answered_correctly'] = model.predict(current_test[features])  # 0.5
#     set_predict(current_test.loc[:,['row_id', 'answered_correctly']])
#     pbar.update(len(current_test))
# print(f"count {count} {len(predicted)}")

In [ ]:
%%time

prior_test_df = None
for (test_df, sample_prediction_df) in iter_test:
    if prior_test_df is not None:
        prior_test_df[target] = eval(test_df['prior_group_answers_correct'].iloc[0])
        prior_test_df = prior_test_df[prior_test_df[target] != -1].reset_index(drop = True)
        
        user_ids = prior_test_df['user_id'].values
        content_ids = prior_test_df['content_id'].values
        targets = prior_test_df[target].values

        # for user_id, content_id, answered_correctly in prior_test_df[["user_id", "content_id", target]].values:
        for user_id, content_id, answered_correctly in zip(user_ids, content_ids, targets):
            user_sum_dict[user_id] += answered_correctly
            user_count_dict[user_id] += 1
            content_sum_dict[content_id] += answered_correctly
            content_count_dict[content_id] += 1

    prior_test_df = test_df.copy()
    
    test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop = True)
    test_df.drop(labels="part", axis=1, inplace=True)
    test_df.content_id = test_df.content_id.astype(int)
    
    test_df = pd.merge(test_df, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].fillna(False).astype('bool')    
    user_sum = np.zeros(len(test_df), dtype = np.int32)
    user_count = np.zeros(len(test_df), dtype = np.int32)
    content_sum = np.zeros(len(test_df), dtype = np.int32)
    content_count = np.zeros(len(test_df), dtype = np.int32)
    
    for i, (user_id, content_id) in enumerate(zip(test_df['user_id'].values, test_df['content_id'].values)):
        user_sum[i] = user_sum_dict[user_id]
        user_count[i] = user_count_dict[user_id]
        content_sum[i] = content_sum_dict[content_id]
        content_count[i] = content_count_dict[content_id]

    test_df['user_correctness'] = user_sum / user_count
    test_df['content_count'] = content_count
    test_df['content_id'] = content_sum / content_count
    test_df[target] = model.predict_proba(test_df[features])[:,1]
    set_predict(test_df[['row_id', target]])

In [ ]:
#!M


In [ ]:
#!M
